# RedNeuronal para la clasificación de ondas de electrocardiograma

## Modelo sin búsqueda de hiperparámetros

In [30]:
# Librerías de TensorFlow/Keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Librerías de Scikit-learn
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Librerías adicionales
import numpy as np
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from scikeras.wrappers import KerasClassifier
import matplotlib.pyplot as plt


In [31]:
# CARGAMOS EL DATASET DE ENTRENAMIENTO
df_train = pd.read_csv("../../data/raw/mitbih_train.csv")

df_train.columns = [f'Feature {i + 1}' for i in range(len(df_train.columns) - 1)] + ['Target']

# SEPARAMOS LOS DATOS PREDICTORES (X) Y OBJETIVO (y)
X_train = df_train.iloc[:, :-1]  # Todas las columnas excepto la última (predictores)
y_train = df_train.iloc[:, -1]   # La última columna (objetivo)

# CONTAMOS LOS VALORES DE CADA CLASE
class_counts = df_train['Target'].value_counts()
print("Distribución de clases antes del submuestreo:")
print(class_counts)

# APLICAMOS RANDOM UNDER SAMPLER PARA BALANCEAR LOS DATOS
undersampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train)


# VERIFICAMOS LA NUEVA DISTRIBUCIÓN DE CLASES
print("Distribución de clases después del submuestreo:")
print(pd.Series(y_resampled).value_counts())

# Oversampling para la clase 1 y 3
oversampler = SMOTE(sampling_strategy={1: 3000, 3: 2000}, random_state=42)
X_over_resampled, y_over_resampled = oversampler.fit_resample(X_train, y_train)

# VERIFICAMOS LA NUEVA DISTRIBUCIÓN DE CLASES
print("Distribución de clases después del submuestreo:")
print(pd.Series(y_over_resampled).value_counts())

Distribución de clases antes del submuestreo:
Target
0.0    72470
4.0     6431
2.0     5788
1.0     2223
3.0      641
Name: count, dtype: int64
Distribución de clases después del submuestreo:
Target
0.0    641
1.0    641
2.0    641
3.0    641
4.0    641
Name: count, dtype: int64
Distribución de clases después del submuestreo:
Target
0.0    72470
4.0     6431
2.0     5788
1.0     3000
3.0     2000
Name: count, dtype: int64


In [32]:
# CARGAMOS EL DATASET DE ENTRENAMIENTO
df_test = pd.read_csv("../../data/raw/mitbih_test.csv")

df_test.columns = [f'Feature {i + 1}' for i in range(len(df_test.columns) - 1)] + ['Target']

# SEPARAMOS LOS DATOS PREDICTORES (X) Y OBJETIVO (y)
X_test = df_test.iloc[:, :-1]  # Todas las columnas excepto la última (predictores)
y_test = df_test.iloc[:, -1]   # La última columna (objetivo)

print(X_test.shape)
print(y_test.shape)

(21891, 187)
(21891,)


In [33]:
# CARGAMOS EL DATASET DE ENTRENAMIENTO
df_normal = pd.read_csv("../../data/raw/ptbdb_normal.csv")
df_normal.columns = [f'Feature {i + 1}' for i in range(len(df_normal.columns) - 1)] + ['Target']


df_abnormal = pd.read_csv("../../data/raw/ptbdb_abnormal.csv")
df_abnormal.columns = [f'Feature {i + 1}' for i in range(len(df_abnormal.columns) - 1)] + ['Target']

ptbd_combined = pd.concat([df_normal, df_abnormal], ignore_index=True)

# SEPARAMOS LOS DATOS PREDICTORES (X) Y OBJETIVO (y)
X_ptbd = ptbd_combined.iloc[:, :-1]  # Todas las columnas excepto la última (predictores)
y_ptbd = ptbd_combined.iloc[:, -1]   # La última columna (objetivo)

print(X_ptbd.shape)
print(y_ptbd.shape)

(14550, 187)
(14550,)


In [5]:
# Imprimir los primeros 5 registros del array 
print(X_train[0:1], X_train.shape)
print()
print()
print(y_train[0:1], y_train.shape)

   Feature 1  Feature 2  Feature 3  Feature 4  Feature 5  Feature 6  \
0   0.960114   0.863248   0.461538   0.196581   0.094017   0.125356   

   Feature 7  Feature 8  Feature 9  Feature 10  ...  Feature 178  Feature 179  \
0   0.099715   0.088319   0.074074    0.082621  ...          0.0          0.0   

   Feature 180  Feature 181  Feature 182  Feature 183  Feature 184  \
0          0.0          0.0          0.0          0.0          0.0   

   Feature 185  Feature 186  Feature 187  
0          0.0          0.0          0.0  

[1 rows x 187 columns] (87553, 187)


0    0.0
Name: Target, dtype: float64 (87553,)


In [12]:
print(X_test[0:1], X_test.shape)
print()
print()
print(y_test[0:1], y_test.shape)

   Feature 1  Feature 2  Feature 3  Feature 4  Feature 5  Feature 6  \
0   0.908425   0.783883   0.531136   0.362637     0.3663   0.344322   

   Feature 7  Feature 8  Feature 9  Feature 10  ...  Feature 178  Feature 179  \
0   0.333333   0.307692   0.296703    0.300366  ...          0.0          0.0   

   Feature 180  Feature 181  Feature 182  Feature 183  Feature 184  \
0          0.0          0.0          0.0          0.0          0.0   

   Feature 185  Feature 186  Feature 187  
0          0.0          0.0          0.0  

[1 rows x 187 columns] (21891, 187)


0    0.0
Name: Target, dtype: float64 (21891,)


In [13]:
print(X_ptbd[0:1], X_ptbd.shape)
print()
print()
print(y_ptbd[0:1], y_ptbd.shape)

   Feature 1  Feature 2  Feature 3  Feature 4  Feature 5  Feature 6  \
0        1.0   0.794681   0.375387   0.116883        0.0   0.171923   

   Feature 7  Feature 8  Feature 9  Feature 10  ...  Feature 178  Feature 179  \
0   0.283859   0.293754   0.325912    0.345083  ...          0.0          0.0   

   Feature 180  Feature 181  Feature 182  Feature 183  Feature 184  \
0          0.0          0.0          0.0          0.0          0.0   

   Feature 185  Feature 186  Feature 187  
0          0.0          0.0          0.0  

[1 rows x 187 columns] (14550, 187)


0    0.0
Name: Target, dtype: float64 (14550,)


In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_ptbd = scaler.transform(X_ptbd)

In [14]:
print(X_train.shape)
print(X_test.shape)
print(X_ptbd.shape)

(87553, 187)
(21891, 187)
(14550, 187)


In [59]:
#y_train_categorical = to_categorical(y_train, num_classes=5) # Cambiar y por y_resampled si se quiere usar el dataset balanceado

model_multiclass = Sequential()
model_multiclass.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
#print(X_train.shape[1])
model_multiclass.add(Dense(64, activation='relu'))
model_multiclass.add(Dense(32, activation='relu'))
model_multiclass.add(Dense(5, activation='softmax'))  # 5 clases en el dataset mitbih

#odel_multiclass.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_multiclass.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model_multiclass.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/10


c:\Users\palmi\.conda\envs\concentracion\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2189/2189 ━━━━━━━━━━━━━━━━━━━━ 3s 954us/step - accuracy: 0.9930 - loss: 0.0318 - val_accuracy: 0.1387 - val_loss: 18.9295
Epoch 2/10
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 2s 912us/step - accuracy: 1.0000 - loss: 2.0503e-06 - val_accuracy: 0.1387 - val_loss: 22.0515
Epoch 3/10
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 2s 885us/step - accuracy: 1.0000 - loss: 2.9702e-07 - val_accuracy: 0.1387 - val_loss: 23.9532
Epoch 4/10
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 2s 900us/step - accuracy: 1.0000 - loss: 7.3512e-08 - val_accuracy: 0.1387 - val_loss: 25.8273
Epoch 5/10
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 2s 885us/step - accuracy: 1.0000 - loss: 1.5113e-08 - val_accuracy: 0.1387 - val_loss: 27.7122
Epoch 6/10
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 2s 916us/step - accuracy: 1.0000 - loss: 5.0087e-09 - val_accuracy: 0.1387 - val_loss: 29.2466
Epoch 7/10
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 3s 915us/step - accuracy: 1.0000 - loss: 1.5948e-09 - val_accuracy: 0.1387 - val_loss: 30.7957
Epoch 8/10
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 2s 881us/step 

In [60]:
model_multiclass.summary()  # Verifica la arquitectura del modelo

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                │ (None, 128)            │        24,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 103,697 (405.07 KB)

 Trainable params: 34,565 (135.02 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 69,132 (270.05 KB)

In [61]:
# Evaluar el modelo
test_loss, test_accuracy = model_multiclass.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

685/685 ━━━━━━━━━━━━━━━━━━━━ 0s 587us/step - accuracy: 0.9836 - loss: 0.6618  
Test Loss: 6.883378982543945
Test Accuracy: 0.827600359916687


In [63]:
# Genrando predicciones y evaluando el modelo
y_pred = model_multiclass.predict(X_ptbd)

# Imprimiendo la predicción para el primer registro y la ultima columna
print(y_pred[0])


import numpy as np
y_pred = np.argmax(y_pred, axis=1)
print(y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_ptbd, y_pred))



455/455 ━━━━━━━━━━━━━━━━━━━━ 0s 582us/step
[1.0000000e+00 1.0722795e-14 1.7546506e-14 2.7008245e-15 2.1319782e-14]
[0 0 0 ... 0 0 0]
              precision    recall  f1-score   support

         0.0       0.28      1.00      0.44      4045
         1.0       0.00      0.00      0.00     10505

    accuracy                           0.28     14550
   macro avg       0.14      0.50      0.22     14550
weighted avg       0.08      0.28      0.12     14550



c:\Users\palmi\.conda\envs\concentracion\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\palmi\.conda\envs\concentracion\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\palmi\.conda\envs\concentracion\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

## Hiperparámetros 

In [106]:
def create_model(num_layers, num_units, activation_function):
    model = Sequential()
    model.add(Dense(num_units, input_dim=X_train.shape[1], activation=activation_function))
    for _ in range(num_layers - 1):
        model.add(Dense(num_units, activation=activation_function))
    model.add(Dense(5, activation='softmax'))
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    #sparse_

    return model

Búsqueda de neuronas por capa y número de capas

In [17]:
hidden_layer_sizes = [(128, 64, 32), 
                      (128, 64, 32, 16),
                      (128, 50, 20),
                      (128, 100, 70, 50, 30)]

clf = GridSearchCV(MLPClassifier(max_iter=5000), {'hidden_layer_sizes': hidden_layer_sizes}, cv = 5)
clf.fit(X_train, y_train)
print(clf.best_estimator_)
print()
print(clf.best_score_)
print()
print(clf.best_params_)
print()
print(clf.best_index_)
print()
print(clf.scorer_)
print()
print(clf.cv_results_)

MLPClassifier(hidden_layer_sizes=(128, 100, 70, 50, 30), max_iter=5000)

0.9806288647282848

{'hidden_layer_sizes': (128, 100, 70, 50, 30)}

3

<class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>.score

{'mean_fit_time': array([343.33007832, 315.41974053, 234.02170796, 466.20570154]), 'std_fit_time': array([108.04059485, 116.10412377,  47.58114849,  49.03625278]), 'mean_score_time': array([0.16529689, 0.15308843, 0.15060511, 0.29266591]), 'std_score_time': array([0.01283441, 0.03436314, 0.03170219, 0.04240189]), 'param_hidden_layer_sizes': masked_array(data=[(128, 64, 32), (128, 64, 32, 16), (128, 50, 20),
                   (128, 100, 70, 50, 30)],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'hidden_layer_sizes': (128, 64, 32)}, {'hidden_layer_sizes': (128, 64, 32, 16)}, {'hidden_layer_sizes': (128, 50, 20)}, {'hidden_layer_sizes': (128, 100, 70, 50, 30)}], 'split0_test_score': array([0.97807093, 0.979

In [18]:


hidden_layer_sizes = [(128, 100, 80, 60, 40, 32),
                      (128, 100, 80, 60, 40, 32, 16),
                      (128, 100, 70, 50, 30)]

clf = GridSearchCV(MLPClassifier(max_iter=5000), {'hidden_layer_sizes': hidden_layer_sizes}, cv = 5)
clf.fit(X_train, y_train)
print(clf.best_estimator_)
print()
print(clf.best_score_)
print()
print(clf.best_params_)
print()
print(clf.best_index_)
print()
print(clf.scorer_)
print()
print(clf.cv_results_)

c:\Users\palmi\.conda\envs\concentracion\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(hidden_layer_sizes=(128, 100, 80, 60, 40, 32, 16), max_iter=5000)

0.9804461400635145

{'hidden_layer_sizes': (128, 100, 80, 60, 40, 32, 16)}

1

<class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>.score

{'mean_fit_time': array([692.19263802, 713.79450908, 489.0308466 ]), 'std_fit_time': array([291.57365088, 262.18377066, 150.38786006]), 'mean_score_time': array([0.55712018, 0.75360341, 0.54598846]), 'std_score_time': array([0.41922651, 0.69971267, 0.43495702]), 'param_hidden_layer_sizes': masked_array(data=[(128, 100, 80, 60, 40, 32),
                   (128, 100, 80, 60, 40, 32, 16), (128, 100, 70, 50, 30)],
             mask=[False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'hidden_layer_sizes': (128, 100, 80, 60, 40, 32)}, {'hidden_layer_sizes': (128, 100, 80, 60, 40, 32, 16)}, {'hidden_layer_sizes': (128, 100, 70, 50, 30)}], 'split0_test_score': array([0.9778425 , 0.97778539, 0.97692879]), 'split1_test_score': arr

In [20]:
clf = clf
clf

GridSearchCV(cv=5, estimator=MLPClassifier(max_iter=5000),
             param_grid={'hidden_layer_sizes': [(128, 100, 80, 60, 40, 32),
                                                (128, 100, 80, 60, 40, 32, 16),
                                                (128, 100, 70, 50, 30)]})

Búsqueda de hiperparámetros

In [15]:
def create_model_hip(optimizer='adam'):
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(80, activation='relu'))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(40, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(5, activation='softmax'))
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [16]:
model = KerasClassifier(build_fn=create_model_hip, epochs=10, batch_size=32, verbose=1)

param_grid = {'batch_size': [32, 64, 128, 256], 'epochs': [30, 50, 70, 100], 'optimizer': ['adam', 'sgd']}

#Imprimiendo el avance de la búsqueda en un for loop
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=3)

grid_result = grid.fit(X_train, y_train)

print(f'Best Accuracy: {grid_result.best_score_} using {grid_result.best_params_}')

Fitting 3 folds for each of 32 candidates, totalling 96 fits


c:\Users\palmi\.conda\envs\concentracion\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\palmi\.conda\envs\concentracion\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
685/685 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8480 - loss: 0.5093
Epoch 2/100
685/685 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9511 - loss: 0.1854
Epoch 3/100
685/685 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9643 - loss: 0.1316
Epoch 4/100
685/685 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9685 - loss: 0.1162
Epoch 5/100
685/685 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9731 - loss: 0.0993
Epoch 6/100
685/685 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9749 - loss: 0.0898
Epoch 7/100
685/685 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9766 - loss: 0.0826
Epoch 8/100
685/685 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9772 - loss: 0.0800
Epoch 9/100
685/685 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9790 - loss: 0.0722
Epoch 10/100
685/685 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9797 - loss: 0.0680
Epoch 11/100
685/685 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9810 - loss: 0.0634
Epoch 12/100
685/685 ━━━━━━━━━━━━━━━━━━━━

In [17]:
# Validando con el dataset de prueba e imprimiendo el resultado
grid_accuracy = grid.score(X_test, y_test)
print(f'Accuracy: {grid_accuracy}')

# Matris de confusión
y_pred = grid.predict(X_test)
confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))


172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.9820930976200265
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 842us/step
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99     18117
         1.0       0.87      0.76      0.81       556
         2.0       0.97      0.94      0.95      1448
         3.0       0.82      0.73      0.77       162
         4.0       0.98      0.99      0.98      1608

    accuracy                           0.98     21891
   macro avg       0.93      0.88      0.90     21891
weighted avg       0.98      0.98      0.98     21891



In [18]:
# Evaluando con test set

best_model = grid_result.best_estimator_
predictions = best_model.predict(X_test)

print(classification_report(y_test, predictions))


172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99     18117
         1.0       0.87      0.76      0.81       556
         2.0       0.97      0.94      0.95      1448
         3.0       0.82      0.73      0.77       162
         4.0       0.98      0.99      0.98      1608

    accuracy                           0.98     21891
   macro avg       0.93      0.88      0.90     21891
weighted avg       0.98      0.98      0.98     21891



## Evaluando modelo con hiperparámetros y tamaño de la red ajustada

Todos los datos

In [12]:
def neural_net(optimizer='adam'):
    model = Sequential()
    model.add(Dense(200, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(80, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(40, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(5, activation='softmax'))
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

### Evaluado con 128,100,80,60,40,32,16

In [36]:
model = KerasClassifier(build_fn=neural_net, epochs=3000, batch_size=128, verbose=3)

neural_model = model.fit(X_train, y_train)


Epoch 1/3000


c:\Users\palmi\.conda\envs\concentracion\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\palmi\.conda\envs\concentracion\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 2/3000
Epoch 3/3000
Epoch 4/3000
Epoch 5/3000
Epoch 6/3000
Epoch 7/3000
Epoch 8/3000
Epoch 9/3000
Epoch 10/3000
Epoch 11/3000
Epoch 12/3000
Epoch 13/3000
Epoch 14/3000
Epoch 15/3000
Epoch 16/3000
Epoch 17/3000
Epoch 18/3000
Epoch 19/3000
Epoch 20/3000
Epoch 21/3000
Epoch 22/3000
Epoch 23/3000
Epoch 24/3000
Epoch 25/3000
Epoch 26/3000
Epoch 27/3000
Epoch 28/3000
Epoch 29/3000
Epoch 30/3000
Epoch 31/3000
Epoch 32/3000
Epoch 33/3000
Epoch 34/3000
Epoch 35/3000
Epoch 36/3000
Epoch 37/3000
Epoch 38/3000
Epoch 39/3000
Epoch 40/3000
Epoch 41/3000
Epoch 42/3000
Epoch 43/3000
Epoch 44/3000
Epoch 45/3000
Epoch 46/3000
Epoch 47/3000
Epoch 48/3000
Epoch 49/3000
Epoch 50/3000
Epoch 51/3000
Epoch 52/3000
Epoch 53/3000
Epoch 54/3000
Epoch 55/3000
Epoch 56/3000
Epoch 57/3000
Epoch 58/3000
Epoch 59/3000
Epoch 60/3000
Epoch 61/3000
Epoch 62/3000
Epoch 63/3000
Epoch 64/3000
Epoch 65/3000
Epoch 66/3000
Epoch 67/3000
Epoch 68/3000
Epoch 69/3000
Epoch 70/3000
Epoch 71/3000
Epoch 72/3000
Epoch 73/3000


In [37]:
neural_accuracy = model.score(X_test, y_test)
print(f'Accuracy: {neural_accuracy}')

# Matris de confusión
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))

Accuracy: 0.9792608834680919
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99     18117
         1.0       0.88      0.74      0.81       556
         2.0       0.95      0.93      0.94      1448
         3.0       0.72      0.75      0.73       162
         4.0       0.99      0.97      0.98      1608

    accuracy                           0.98     21891
   macro avg       0.90      0.88      0.89     21891
weighted avg       0.98      0.98      0.98     21891



### Evaluado con 200,128,100,80,60,40,32,16

In [35]:
model = KerasClassifier(build_fn=neural_net, epochs=1000, batch_size=128, verbose=3)

neural_model = model.fit(X_train, y_train)

Epoch 1/1000


c:\Users\palmi\.conda\envs\concentracion\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\palmi\.conda\envs\concentracion\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 2/1000
Epoch 3/1000
Epoch 4/1000
Epoch 5/1000
Epoch 6/1000
Epoch 7/1000
Epoch 8/1000
Epoch 9/1000
Epoch 10/1000
Epoch 11/1000
Epoch 12/1000
Epoch 13/1000
Epoch 14/1000
Epoch 15/1000
Epoch 16/1000
Epoch 17/1000
Epoch 18/1000
Epoch 19/1000
Epoch 20/1000
Epoch 21/1000
Epoch 22/1000
Epoch 23/1000
Epoch 24/1000
Epoch 25/1000
Epoch 26/1000
Epoch 27/1000
Epoch 28/1000
Epoch 29/1000
Epoch 30/1000
Epoch 31/1000
Epoch 32/1000
Epoch 33/1000
Epoch 34/1000
Epoch 35/1000
Epoch 36/1000
Epoch 37/1000
Epoch 38/1000
Epoch 39/1000
Epoch 40/1000
Epoch 41/1000
Epoch 42/1000
Epoch 43/1000
Epoch 44/1000
Epoch 45/1000
Epoch 46/1000
Epoch 47/1000
Epoch 48/1000
Epoch 49/1000
Epoch 50/1000
Epoch 51/1000
Epoch 52/1000
Epoch 53/1000
Epoch 54/1000
Epoch 55/1000
Epoch 56/1000
Epoch 57/1000
Epoch 58/1000
Epoch 59/1000
Epoch 60/1000
Epoch 61/1000
Epoch 62/1000
Epoch 63/1000
Epoch 64/1000
Epoch 65/1000
Epoch 66/1000
Epoch 67/1000
Epoch 68/1000
Epoch 69/1000
Epoch 70/1000
Epoch 71/1000
Epoch 72/1000
Epoch 73/1000


In [36]:
neural_accuracy = model.score(X_test, y_test)
print(f'Accuracy: {neural_accuracy}')

# Matris de confusión
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))

Accuracy: 0.9811337992782422
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99     18117
         1.0       0.85      0.76      0.81       556
         2.0       0.95      0.95      0.95      1448
         3.0       0.77      0.73      0.75       162
         4.0       0.99      0.98      0.99      1608

    accuracy                           0.98     21891
   macro avg       0.91      0.88      0.90     21891
weighted avg       0.98      0.98      0.98     21891



## Evaluando con UnderSampling

In [13]:
print(X_resampled.shape, y_resampled.shape)

(3205, 187) (3205,)


In [14]:
model = KerasClassifier(build_fn=neural_net, epochs=500, batch_size=64, verbose=3)
neural_model = model.fit(X_resampled, y_resampled)


c:\Users\palmi\.conda\envs\concentracion\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\palmi\.conda\envs\concentracion\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/500
Epoch 2/500
Epoch 3/500
Epoch 4/500
Epoch 5/500
Epoch 6/500
Epoch 7/500
Epoch 8/500
Epoch 9/500
Epoch 10/500
Epoch 11/500
Epoch 12/500
Epoch 13/500
Epoch 14/500
Epoch 15/500
Epoch 16/500
Epoch 17/500
Epoch 18/500
Epoch 19/500
Epoch 20/500
Epoch 21/500
Epoch 22/500
Epoch 23/500
Epoch 24/500
Epoch 25/500
Epoch 26/500
Epoch 27/500
Epoch 28/500
Epoch 29/500
Epoch 30/500
Epoch 31/500
Epoch 32/500
Epoch 33/500
Epoch 34/500
Epoch 35/500
Epoch 36/500
Epoch 37/500
Epoch 38/500
Epoch 39/500
Epoch 40/500
Epoch 41/500
Epoch 42/500
Epoch 43/500
Epoch 44/500
Epoch 45/500
Epoch 46/500
Epoch 47/500
Epoch 48/500
Epoch 49/500
Epoch 50/500
Epoch 51/500
Epoch 52/500
Epoch 53/500
Epoch 54/500
Epoch 55/500
Epoch 56/500
Epoch 57/500
Epoch 58/500
Epoch 59/500
Epoch 60/500
Epoch 61/500
Epoch 62/500
Epoch 63/500
Epoch 64/500
Epoch 65/500
Epoch 66/500
Epoch 67/500
Epoch 68/500
Epoch 69/500
Epoch 70/500
Epoch 71/500
Epoch 72/500
Epoch 73/500
Epoch 74/500
Epoch 75/500
Epoch 76/500
Epoch 77/500
Epoch 78

In [17]:
neural_accuracy = model.score(X_resampled, y_resampled)
print(f'Accuracy: {neural_accuracy}')

# Matris de confusión
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))

Accuracy: 0.9909516380655227
              precision    recall  f1-score   support

         0.0       0.99      0.85      0.91     18117
         1.0       0.20      0.80      0.33       556
         2.0       0.80      0.85      0.82      1448
         3.0       0.18      0.91      0.30       162
         4.0       0.87      0.96      0.91      1608

    accuracy                           0.86     21891
   macro avg       0.61      0.87      0.65     21891
weighted avg       0.94      0.86      0.89     21891



## Evaluando con Oversampling

In [18]:
print(X_over_resampled.shape, y_over_resampled.shape)

(89689, 187) (89689,)


In [19]:
model = KerasClassifier(build_fn=neural_net, epochs=500, batch_size=64, verbose=3)
neural_model = model.fit(X_over_resampled, y_over_resampled)


Epoch 1/500


c:\Users\palmi\.conda\envs\concentracion\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\palmi\.conda\envs\concentracion\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 2/500
Epoch 3/500
Epoch 4/500
Epoch 5/500
Epoch 6/500
Epoch 7/500
Epoch 8/500
Epoch 9/500
Epoch 10/500
Epoch 11/500
Epoch 12/500
Epoch 13/500
Epoch 14/500
Epoch 15/500
Epoch 16/500
Epoch 17/500
Epoch 18/500
Epoch 19/500
Epoch 20/500
Epoch 21/500
Epoch 22/500
Epoch 23/500
Epoch 24/500
Epoch 25/500
Epoch 26/500
Epoch 27/500
Epoch 28/500
Epoch 29/500
Epoch 30/500
Epoch 31/500
Epoch 32/500
Epoch 33/500
Epoch 34/500
Epoch 35/500
Epoch 36/500
Epoch 37/500
Epoch 38/500
Epoch 39/500
Epoch 40/500
Epoch 41/500
Epoch 42/500
Epoch 43/500
Epoch 44/500
Epoch 45/500
Epoch 46/500
Epoch 47/500
Epoch 48/500
Epoch 49/500
Epoch 50/500
Epoch 51/500
Epoch 52/500
Epoch 53/500
Epoch 54/500
Epoch 55/500
Epoch 56/500
Epoch 57/500
Epoch 58/500
Epoch 59/500
Epoch 60/500
Epoch 61/500
Epoch 62/500
Epoch 63/500
Epoch 64/500
Epoch 65/500
Epoch 66/500
Epoch 67/500
Epoch 68/500
Epoch 69/500
Epoch 70/500
Epoch 71/500
Epoch 72/500
Epoch 73/500
Epoch 74/500
Epoch 75/500
Epoch 76/500
Epoch 77/500
Epoch 78/500
Epoch 7

In [20]:
neural_accuracy = model.score(X_over_resampled, y_over_resampled)
print(f'Accuracy: {neural_accuracy}')

# Matris de confusión
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))

Accuracy: 0.9982272073498423
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99     18117
         1.0       0.86      0.75      0.80       556
         2.0       0.97      0.93      0.95      1448
         3.0       0.75      0.84      0.79       162
         4.0       0.99      0.98      0.98      1608

    accuracy                           0.98     21891
   macro avg       0.91      0.90      0.90     21891
weighted avg       0.98      0.98      0.98     21891



In [29]:
y_pred = model.predict(X_ptbd)
print(classification_report(y_ptbd, y_pred))
print(confusion_matrix(y_ptbd, y_pred))

              precision    recall  f1-score   support

         0.0       0.30      0.96      0.45      4045
         1.0       0.84      0.04      0.07     10505
         2.0       0.00      0.00      0.00         0
         3.0       0.00      0.00      0.00         0
         4.0       0.00      0.00      0.00         0

    accuracy                           0.29     14550
   macro avg       0.23      0.20      0.11     14550
weighted avg       0.69      0.29      0.18     14550

[[3884   79   80    0    2]
 [9241  405  489   40  330]
 [   0    0    0    0    0]
 [   0    0    0    0    0]
 [   0    0    0    0    0]]


c:\Users\palmi\.conda\envs\concentracion\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\palmi\.conda\envs\concentracion\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\palmi\.conda\envs\concentracion\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Obteniendo características para modelo binario

In [16]:

# Accede a la penúltima capa por nombre
intermediate_layer = model_multiclass.get_layer('dense_6')  # Nombre de la penúltima capa

# Crea un nuevo modelo que termina en la penúltima capa
intermediate_layer_model = Model(inputs=model_multiclass.input, outputs=intermediate_layer.output)

# Genera características usando el modelo intermedio
X_test_features = intermediate_layer_model.predict(X_test)
X_ptbd_features = intermediate_layer_model.predict(X_ptbd)

print("Características extraídas de X_test:", X_test_features.shape)
print("Características extraídas de X_ptbd:", X_ptbd_features.shape)

ValueError: The layer sequential_1 has never been called and thus has no defined input.

In [ ]:
model_binary = Sequential()
model_binary.add(Dense(128, input_dim=X_ptbd_features.shape[1], activation='relu'))
model_binary.add(Dense(64, activation='relu'))
model_binary.add(Dense(32, activation='relu'))
model_binary.add(Dense(1, activation='sigmoid'))  # Salida binaria

model_binary.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_binary.fit(X_ptbd_features, y_ptbd, epochs=20, batch_size=32, validation_split=0.2, verbose=1)


In [ ]:
model_binary.summary()

In [ ]:
# Evaluar el modelo binario
binary_test_loss, binary_test_accuracy = model_binary.evaluate(X_ptbd_features, y_ptbd, verbose=1)
print(f'Test Loss (Binario): {binary_test_loss}')
print(f'Test Accuracy (Binario): {binary_test_accuracy}')